In [1]:
%pip install cmem-cmempy llama-index python-dotenv llama-index-vector-stores-postgres

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os

import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
!docker pull ankane/pgvector
!docker run -d -e POSTGRES_USER=eccenca -e POSTGRES_PASSWORD=eccenca -e POSTGRES_DB=eccenca --name pgvector-eccenca -p 5432:5432 ankane/pgvector

Using default tag: latest
latest: Pulling from ankane/pgvector

cdd3a2a4: Pulling fs layer 
234636c9: Pulling fs layer 
2fa63f18: Pulling fs layer 
e54346e0: Pulling fs layer 
7c0dd616: Pulling fs layer 
e7c4b030: Pulling fs layer 
3abf3802: Pulling fs layer 
3b657e91: Pulling fs layer 
23549f11: Pulling fs layer 
57d7d7b0: Pulling fs layer 
dba297eb: Pulling fs layer 
7b8030eb: Pulling fs layer 
9b500022: Pulling fs layer 
cf30f429: Pulling fs layer 
57d7d7b0: Download complete MB/104.3MB12ADigest: sha256:956744bd14e9cbdf639c61c2a2a7c7c2c48a9c8cdd42f7de4ac034f4e96b90f8
Status: Downloaded newer image for ankane/pgvector:latest
docker.io/ankane/pgvector:latest
96a9465720137f2fd7cc29e12a3fb68a7a18a782ff10af02498e5459d2fc8424


In [5]:
from IPython.display import Markdown

from llama_index_cmem.readers.cmem import CMEMReader

reader = CMEMReader()
documents = reader.load_data(graph="<http://ld.company.org/prod-inst/>")
display(Markdown(f"### Documents loaded: __{len(documents)}__ "))

### Documents loaded: __20043__ 

In [6]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database="eccenca",
    host="localhost",
    password="eccenca",
    port="5432",
    user="eccenca",
    table_name="prod-inst",
    embed_dim=1536,  # openai embedding dimension
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, show_progress=True
)
query_engine = index.as_query_engine()

/home/pfreytag/github.com/eccenca/llama-index-cmem/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 2048/2048 [00:25<00:00, 79.24it/s]
PG Setup: Error creating HNSW index: (psycopg2.errors.SyntaxError) syntax error at or near "-"
LINE 1: CREATE INDEX IF NOT EXISTS data_prod-inst_embedding_idx ON p...
                                            ^

[SQL: CREATE INDEX IF NOT EXISTS data_prod-inst_embedding_idx ON public.data_prod-inst USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64)]
(Background on this error at: https://sqlalche.me/e/20/f405)
Generating embeddings: 100%|██████████| 1611/1611 [00:18<00:00, 87.36it/s]


In [7]:
question = "What information do you have about Liese Adam? Give me as many details as possible."
answer = query_engine.query(question)
display(Markdown(f"## Question: _{question}_"))
display(Markdown(f"### Answer:\n\n{answer}"))

## Question: _What information do you have about Liese Adam? Give me as many details as possible._

### Answer:

Liese Adam's name is Liese Adam and her email address is Liese.Adam@company.org.